In [6]:
import sys
sys.path.insert(1, '../') 
from utils.loader import Loader
import tensorflow as tf
import pandas as pd

In [7]:
train = Loader.load_train()

In [8]:
descriptions = [col for col in train.columns if "description" in col]

train = train[descriptions]
train["Description"] = 

descriptions.append("idx")
descriptions.append("price")
descriptions.append("km")

,motor_description,transmission_description,parking_sensors_description,Exterior_Llantas_description,Exterior_Molduras_description,Exterior_Neumáticos_description,Exterior_Puerta_description,Exterior_Retrovisor_description,Confort_Volante_description,seat_description,head_supporter_description,idx,price,km
0,"Motor de 1,3 litros ( 1.332 cc ) , cuatro cili...",Transmisión de tipo manual con cambio totalmen...,Sensores de aparcamiento delanteros con radar ...,llantas delanteras traseras aleación ligera 19...,no tiene,neumáticos delanteros traseros 19 pulgadas dia...,"puerta conductor, trasera (lado conductor), pa...",retrovisor exterior conductor acompañante alea...,Volante multi-función de aleación y cuero ajus...,Asiento delantero del conductor y acompañante ...,Dos reposacabezas en asientos delanteros ajust...,105953,22490,68887.0
1,"Motor de 2,0 litros ( 1.969 cc ) , cuatro cili...",Transmisión de tipo automático con cambio auto...,No tiene,llantas delanteras traseras aleación ligera 17...,no tiene,neumáticos delanteros traseros 17 pulgadas dia...,"puerta conductor, trasera (lado conductor), pa...",retrovisor exterior conductor acompañante colo...,Volante multi-función de aleación y cuero ajus...,"Asiento delantero del conductor individual, aj...",Dos reposacabezas en asientos delanteros ajust...,15683,19990,116892.0
2,"Motor de 1,6 litros ( 1.560 cc ) , cuatro cili...",Transmisión de tipo manual con cambio totalmen...,No tiene,llantas delanteras traseras acero 16 pulgadas ...,molduras laterales,neumáticos delanteros traseros 16 pulgadas dia...,puerta conductor pasajero bisagras delanteras,retrovisor exterior conductor acompañante ajus...,Volante en material plástico ajustable en altu...,Asiento delantero del conductor y acompañante ...,Dos reposacabezas en asientos delanteros ajust...,153684,17500,130820.0
3,"Motor de 1,8 litros ( 1.798 cc ) , cuatro cili...",Transmisión de tipo automático con cambio de v...,Sensores de aparcamiento traseros con cámara,llantas delanteras traseras aleación ligera 17...,no tiene,neumáticos delanteros traseros 17 pulgadas dia...,"puerta conductor, trasera (lado conductor), pa...",retrovisor exterior conductor acompañante pint...,Volante multi-función revestido de cuero ajust...,Asiento delantero del conductor y acompañante ...,Dos reposacabezas en asientos delanteros ajust...,156928,22900,50773.0
4,"Motor de 1,9 litros ( 1.910 cc ) , 4 cilindros...",NaN,No tiene,"llantas delanteras traseras aleación ligera, 1...",no tiene,"neumáticos delanteros, traseros (medida oficia...","puerta conductor, trasera (lado conductor), pa...",retrovisor exterior lado conductor acompañante...,Volante multifuncional de aleación y cuero aju...,Asiento delantero del conductor individual con...,Dos reposacabezas activos en F ajustables en a...,24443,4000,269000.0
